### 6. Working with financial data

Before importing the data, first some package imports:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

First let's try a built-in Python function to import a CSV file (https://docs.python.org/3/library/functions.html#open) :

In [2]:
#Enter the path where you saved the price_data.csv file
filename = r'C:\Users\r.skripnik\Desktop\Python-R\price_data.csv'
f = open(filename, 'r')
f.readlines()[:5]

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\r.skripnik\\Desktop\\Python-R\\price_data.csv'

It is not convenient working with this data. Fortunately, **pandas** provides a number of different functions and **DataFrame** methods to import data stored in different formats (CSV, SQL, Excel, etc.) and to export data to different formats. The following code uses the **pd.read_csv()** function to import the time series data set from the CSV file (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) :

In [ ]:
data = pd.read_csv(filename,
                   index_col=0,
                   parse_dates=True)
data.info()

The data used is from YAHOO! finance. Here:

* AAPL - Apple stock;
* AMZN - Amazon stock;
* GDX - VanEck Vectors Gold Miners ETF;
* MSFT - Microsoft stock;
* SPY - SPDR S&P 500 ETF;
* VIX - Volatility index.

The **DataFrame** contains daily adjusted closing prices for the above instruments.

In [ ]:
data.head()

In [ ]:
data.plot(figsize=(10, 12),
          subplots=True)

A next step the financial analyst might take is to have a look at different summary statistics for the data (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html) :

In [ ]:
data.describe().round(2)

There are also options, of course, to customize what type of statistic to derive and display. We can do it with simple methods, such as **.mean()** (and others) or choose desired statistics and display them all at once, using **.aggregate()** (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.aggregate.html) method:

In [ ]:
data.mean()

In [ ]:
data.aggregate([np.min,
                np.mean,
                np.std,
                np.median,
                np.max]).round(2)

Most statistical analysis methods are based on changes of a time series over time and not the absolute values themselves. There are multiple options to calculate the changes of a time series over time, such as absolute differences, percentage changes and logarithmic returns. First, the absolute differences for which **pandas** provides a special **.diff()** (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.diff.html) method:

In [ ]:
data.diff().head()

In [ ]:
data.diff().mean()

From a statistics point of view, absolute changes are not optimal because they are dependent on the scale of the time series data itself. Therefore, percentage changes are usually preferred. The following code derives the percentage changes using **.pct_change()** (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pct_change.html) method and visualizes their mean values per column:

In [ ]:
data.pct_change().round(4).head()

In [ ]:
data.pct_change().mean().plot(kind='bar',
                              figsize=(10, 6))

As an alternative to percentage returns, log returns can be used. In some scenarios, they are more easy to handle and therefore often preferred in a financial context. To calculate log returns, we need to first lag the data (shift prices one period back) using **.shift()** method (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.shift.html) .

In [ ]:
data.head()

In [ ]:
data.shift(1).head()

In [ ]:
rets = np.log(data / data.shift(1))
rets.head()

Since we have calculated log values, we need to convert them back using exponent function. The cumulative log returns for the financial times series:

In [ ]:
rets.cumsum().apply(np.exp).plot(figsize=(10, 6), grid=True)

Resampling is an important operation on financial time series data. Usually, this takes on the form of up-sampling, meaning that, for example, a time series with daily observations is resampled to a time series with weekly or monthly observations. In **pandas** it is done using **.resample()** method (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.resample.html). Let's see how to resample with weekly and monthly frequencies, and how to plot monthly data:

In [ ]:
data.resample('1w').last().head()

In [ ]:
data.resample('1m').last().head()

In [ ]:
rets.cumsum().apply(np.exp).resample('1m').last().plot(figsize=(10, 6), grid=True)

It is financial tradition to work with rolling statistics, often also called financial indicators or financial studies. Such rolling statistics are basic tools for financial chartists and technical traders. This section uses **.rolling()** (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html) method and works with a single financial time series only:

In [ ]:
sym = 'GDX'
new_data = pd.DataFrame(data[sym]).dropna()
new_data.head()

In [ ]:
window = 20
new_data['Min'] = new_data[sym].rolling(window=window).min()

new_data['Mean'] = new_data[sym].rolling(window=window).mean()

new_data['Std'] = new_data[sym].rolling(window=window).std()

new_data['Median'] = new_data[sym].rolling(window=window).median()

new_data['Max'] = new_data[sym].rolling(window=window).max()

new_data['Ewma'] = new_data[sym].ewm(halflife=0.5,
                                     min_periods=window).mean()
new_data

In [ ]:
ax = new_data[['Min', 'Mean', 'Max']].iloc[-200:].plot(figsize=(10, 6),
                                                       style=['g--', 'r--', 'g--'],
                                                       lw=0.8)
new_data[sym].iloc[-200:].plot(ax=ax,
                               lw=2.0)

Rolling statistics are a major tool in the technical analysis of stocks as compared to the fundamental analysis which focuses on financial reports and the strategic positions of the company. A decades-old trading strategy based on technical analysis uses two simple moving averages (SMAs). The idea is that the trader should be long a stock when the shorter-term SMA is above the longer-term SMA and should be short the stock when the opposite holds true. The concepts can be made precise with **pandas** and the capabilities of the **DataFrame** object. Rolling statistics are generally only calculated when there is enough data given the **window** parameter specification. The SMA time series only start at the day for which there is enough data given the specific parametrization:

In [ ]:
new_data['SMA1'] = new_data[sym].rolling(window=42).mean()
new_data['SMA2'] = new_data[sym].rolling(window=252).mean()
new_data[[sym, 'SMA1', 'SMA2']].tail()

In [ ]:
new_data[[sym, 'SMA1', 'SMA2']].plot(figsize=(10, 6))

In this context, the SMAs are only a means to an end. They are used to derive positionings to implement a trading strategy. In the following, long position is visualized by a value of 1 and a short position by a value of -1. The change in the position is triggered (visually) by a crossover of the two lines representing the SMA time series (https://numpy.org/doc/stable/reference/generated/numpy.where.html) :

In [ ]:
new_data.dropna(inplace=True)
new_data

In [ ]:
new_data['Positions'] = np.where(new_data['SMA1'] > new_data['SMA2'],
                                 1,
                                 -1)
new_data

In [ ]:
ax = new_data[[sym, 'SMA1', 'SMA2', 'Positions']].plot(figsize=(10, 6),
                                                       secondary_y='Positions')

The trading strategy implicitly derived here only leads to a few trades: only when the position value changes (i.e. a crossover happens), a trade takes place. Including opening and closing trades, this would add up to six trades only in total.

The following data set consists of two financial times series:

In [ ]:
#data = pd.read_csv(filename,
#                  index_col=0,
#                  parse_dates=True)

data = data[['SPY', '^VIX']].dropna()
data

In [ ]:
data.plot(figsize=(10, 6),
          subplots=True)

When plotting (parts of) the two time series in a single plot and with adjusted scalings, the stylized fact of negative correlation between the two indices becomes already evident through simple visual inspection (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.html) :

In [ ]:
data.loc['2018-01-01':].plot(figsize=(10, 6),
                             secondary_y='^VIX')

As pointed out above, statistical analysis in general relies on returns instead of absolute changes or even absolute values. Therefore, the calculation of log returns first before any further analysis takes place. The following shows the high variability of the log returns over time. For both indices so-called volatility clusters can be spotted. And in general, periods of high volatility in the stock index are accompanied by the same phenomenon in the volatility index:

In [ ]:
rets = np.log(data / data.shift(1))
rets.head()

In [ ]:
rets.dropna(inplace=True)
rets.plot(figsize=(10, 6),
          subplots=True)

In such a context, the **pandas** **.plotting.scatter_matrix()** (https://pandas.pydata.org/docs/reference/api/pandas.plotting.scatter_matrix.html) plotting function comes in handy for visualizations. It plots the log returns of the two series against each other:

In [ ]:
pd.plotting.scatter_matrix(rets,
                           figsize=(10, 6),
                           diagonal='hist',
                           hist_kwds={'bins': 35})

With all these preparations, an ordinary least-squares (OLS) regression analysis can be implemented. **numpy.polyfit()** (https://numpy.org/doc/stable/reference/generated/numpy.polyfit.html) function is used to estimate the coefficients (intercept and the slope coefficient), and **numpy.polyval()** (https://numpy.org/doc/stable/reference/generated/numpy.polyval.html) function is used to predict the values.

In [ ]:
coeffs = np.polyfit(rets['SPY'],
                    rets['^VIX'],
                    deg=1)
coeffs

In [ ]:
pred = np.polyval(coeffs,
                  rets['SPY'])
pred

The following shows a scatter plot of the log returns and the linear regression line through the cloud of dots. The slope is obviously negative providing support for the stylized fact about the negative correlation between the two indices:

In [ ]:
ax = rets.plot(figsize=(10, 6),
               kind='scatter',
               x='SPY',
               y='^VIX')

ax.plot(rets['SPY'],
        pred,
        'r',
        lw=2)

Finally, consider correlation measures directly. Two such measures are considered: a static one taking into account the complete data set and a rolling one showing the correlation for a fixed window over time. The following illustrates that the correlation indeed varies over time, but it is always negative. I use **.axhline()** (https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.axhline.html) method to plot a horizontal line.

In [ ]:
rets.corr()

In [ ]:
ax = rets['SPY'].rolling(window=252).corr(rets['^VIX']).dropna().plot(figsize=(10, 6))

ax.axhline(rets.corr().iloc[0, 1],
           c='r')

This provides strong support for the stylized fact that the S&P 500 and the VIX indices are strongly negatively correlated.

One of the major strengths of **pandas** is that it can read and write different data formats natively, among others, including:

* CSV (comma-separated value);
* SQL (Structured Query Language);
* XLS/XSLX (Microsoft Excel files);
* JSON (JavaScript Object Notation);
* HTML (HyperText Markup Language).

Let's export some data:

In [ ]:
np.random.seed(6)
data = np.random.standard_normal((10000, 5)).round(4)
data = pd.DataFrame(data,
                    columns=['Col1', 'Col2', 'Col3', 'Col4', 'Col5'])
data.head()

One of the most widely used formats to exchange financial data is the **CSV** format. Although it is not really standardized, it can be processed by any platform and the vast majority of applications. **pandas** makes this whole procedure convenient:

In [ ]:
directory = r'C:\Users\r.skripnik\Desktop\Python-R\\'
data.to_csv(directory + 'sample.csv')

df = pd.read_csv(directory + 'sample.csv',
                 index_col=0)
df.head()

In [ ]:
df[['Col1', 'Col2', 'Col3', 'Col4']].hist(figsize=(10, 6),
                                          bins=20)

The following code briefly demonstrates how **pandas** can write data in **Excel** format and read data from **Excel** spreadsheets. Here only 5,000 first rows are used:

In [ ]:
data[:5000].to_excel(directory + 'sample.xlsx')

df = pd.read_excel(directory + 'sample.xlsx',
                   'Sheet1',
                   index_col=0)

df.cumsum().plot(figsize=(10, 6))

Generating the **Excel** spreadsheet file with a smaller subset of the data takes quite a while. This illustrates what kind of overhead the spreadsheet structure brings along with it.

**_Exercises._**

Exercise 1. Assume you invested 1000 USD in each of MSFT, AMZN stocks and GDX for the entire period (2016-2019). Plot the value of the portfolio over time (daily), together with the value of SPY (assuming you invested 4000 USD). Indicate in the plot when the value of your portfolio was above the value of SPY investment and when it was below.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

data = yf.download('AMZN MSFT SPY GDX', start = '2016-01-01', end = '2019-12-31', period = '1d')
prices = data['Adj Close']
ratios = prices / prices.iloc[0]
ratios

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

data = yf.download('AMZN MSFT SPY GDX', start = '2016-01-01', end = '2019-12-31', period = '1d')
prices = data['Adj Close']
ratios = prices / prices.iloc[0]

ratios['Portfolio'] = (ratios['AMZN'] + ratios['GDX'] + ratios['MSFT']) * 1000
ratios['SPY_port'] = ratios['SPY'] * 4000

ratios['Positions'] = np.where(ratios['Portfolio'] > ratios['SPY_port'],
                               1,
                               -1)

ratios[['Portfolio', 'SPY_port', 'Positions']].plot(figsize=(10, 6),
                                                    secondary_y='Positions')

Exercise 2. In one chart plot two sets of scatterplots: one for SPY returns against VIX returns and one for SPY returns against AAPL returns (resample weekly). Fit a line through each of the cloud of dots.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

data = yf.download('AAPL SPY ^VIX', start = '2016-01-01', end = '2019-12-31', period = '1d')
prices = data['Adj Close']
prices = prices.resample('1w').last()
rets = np.log(prices / prices.shift(1))
rets.dropna(inplace=True)

coeffs_1 = np.polyfit(rets['SPY'],
                      rets['^VIX'],
                      deg=1)
pred_1 = np.polyval(coeffs_1,
                    rets['SPY'])

coeffs_2 = np.polyfit(rets['SPY'],
                      rets['AAPL'],
                      deg=1)
pred_2 = np.polyval(coeffs_2,
                    rets['SPY'])

fig = plt.figure(figsize=(10, 6))
plt.scatter(rets['SPY'], rets['^VIX'], c='b')
plt.plot(rets['SPY'], pred_1, 'b')

plt.scatter(rets['SPY'], rets['AAPL'], c='r')
plt.plot(rets['SPY'], pred_2, 'r')

Exercise 3. Plot Stochastic Oscillator:

$K = (\frac{C - L14}{H14 - L14})$

C - the most recent closing price

L14 - the lowest price traded of the 14 previous trading days

H14 - the highest price traded of the 14 previous trading days

In [ ]:
import yfinance as yf
yf.download('SPY', start = '2019-01-01', end = '2019-12-31', period = '1d')

In [ ]:
import yfinance as yf

SPY = yf.download('SPY', start = '2019-01-01', end = '2019-12-31', period = '1d')

window = 14
SPY['L14'] = SPY['Low'].rolling(window=window).min()
SPY['H14'] = SPY['High'].rolling(window=window).max()

SPY['Oscillator'] = (SPY['Close'] - SPY['L14']) / (SPY['H14'] - SPY['L14'])
ax = SPY['Oscillator'].plot(figsize=(10, 6),
                            title='Stochastic Oscillator')

ax.axhline(0.8,
           c='r',
           ls='--')

ax.axhline(0.2,
           c='r',
           ls='--')